4/25/19
Pandas on ccs and lima score report csv files
Main goal: combine the two csv files, with the rows (indices) being the ZMW hole numbers

In [2]:
import numpy as np
import pandas as pd
df_ccs = pd.read_csv('/Users/catherineanderson/Desktop/bam_table.csv')
df_ccs.set_index('ZMW hole number')

df_lima = pd.read_csv('/Users/catherineanderson/Desktop/score_report_span_0.csv')
df_lima.set_index('ZMW hole number')



a = 1



In [3]:
df_combo = pd.concat([df_ccs, df_lima],sort = False)#,keys = ['ZMW hole number'], sort = False)
#df_combo.set_index('ZMW hole number')
a = 1

In [4]:
df_combo.set_index('ZMW hole number')
a = 1

In [5]:
df_combo.to_csv('/Users/catherineanderson/Documents/Research_Spring_2019/ccs_lima_combo.csv', index = False)

In [ ]:
#df_combo